In [0]:
tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@earthquakedata.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 
# Get base parameters
dbutils.widgets.text("start_date", "")
dbutils.widgets.text("end_date", "")
start_date = dbutils.widgets.get("start_date")
end_date = dbutils.widgets.get("end_date")

     

In [0]:
import requests

# Step 1: API URL
API_URL = "https://earthquake.usgs.gov/fdsnws/event/1/query"
PARAMS = {
    "format": "geojson",
    "starttime": start_date,
    "endtime": end_date
}

# Step 2: Fetch raw data from API
try:
    response = requests.get(API_URL, params=PARAMS, timeout=10)
    response.raise_for_status()
    raw_json = response.text  # Keep the raw JSON as string
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    raw_json = None


# Step 3: Define ADLS Gen2 path
output_path = f"abfss://bronze@earthquakedata.dfs.core.windows.net/earthquake_data_{start_date}.json"
# Step 4: Write raw JSON to ADLS
if raw_json:
    # Use Databricks dbutils to write directly
    dbutils.fs.put(output_path, raw_json, overwrite=True)
    print(f"Raw JSON data successfully written to {output_path}")
else:
    print("No data to write.")


Wrote 169274 bytes.
Raw JSON data successfully written to abfss://bronze@earthquakedata.dfs.core.windows.net/earthquake_data_2025-12-21.json


In [0]:
import json
# Define your variables
output_data = {
    "start_date": start_date,
    "end_date": end_date,
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Serialize the dictionary to a JSON string
output_json = json.dumps(output_data)

# Log the serialized JSON for debugging
print(f"Serialized JSON: {output_json}")

# Return the JSON string
dbutils.notebook.exit(output_json)
